In [1]:
import os
if os.path.exists('travel_tmp.sqlite'):
    os.remove('travel_tmp.sqlite')
else:
    pass

In [2]:
# Import SQL Alchemy
from sqlalchemy import create_engine, func

# Import and establish Base for which classes will be constructed 
from sqlalchemy.orm import declarative_base, Session
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Boolean

In [3]:
# Create the Travel class
class Travel(Base):
    __tablename__ = 'travel_destinations'
    id = Column(Integer, primary_key=True)
    city = Column(String)
    country = Column(String)
    distance = Column(Float)
    budget = Column(Float)
    visited = Column(Boolean)


In [4]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///travel_tmp.sqlite')

In [5]:
# Create the travel_destinations table within the database
Base.metadata.create_all(engine)

In [6]:
# To push the objects made and query the server we use a Session object
s = Session(bind=engine)

In [7]:
# Create some instances of the Travel class
# Add these objects to the session
# Commit the objects to the database
s.add_all([
    Travel(city='Ubud', country='Indonesia', distance=10200.0, budget=4000.0, visited=True),
    Travel(city='Swakopmund', country='Namibia', distance=7200.0, budget=5000.0, visited=True),
    Travel(city='Ollantaytambo', country='Peru', distance=3600.0, budget=3000.0, visited=True),
    Travel(city='Maastricht', country='Netherlands', distance=3900.0, budget=4000.0, visited=True),
    Travel(city='Burlington', country='USA', distance=600.0, budget=200.0, visited=True)
])
s.commit()

In [8]:
# Update two rows of data
bsq = s.query(Travel).filter_by(city='Burlington')
bsq.distance = 500.0

msq = s.query(Travel).filter_by(city='Maastricht')
msq.budget = 4500.0

# Commit the updates to the database
s.commit()


In [9]:
# Delete the row with the shortest distance
# take the .first() one to delete, which will be the shortest distance
axed = s.query(Travel).order_by(Travel.distance).first()

# Commit the delete to the database
s.delete(axed)


In [10]:
# Collect all of the destinations and print their information
sq = s.query(Travel).all()
for _ in sq:
    print(_.id, _.city, _.country, _.distance, _.budget, _.visited, sep=', ')


1, Ubud, Indonesia, 10200.0, 4000.0, True
2, Swakopmund, Namibia, 7200.0, 5000.0, True
3, Ollantaytambo, Peru, 3600.0, 3000.0, True
4, Maastricht, Netherlands, 3900.0, 4000.0, True


In [11]:
# Close the session
s.close()

In [12]:
# BONUS
# Use a SQLAlchemy function to identify the item with the lowest id value, and change its budget to $2,500.
s = Session(bind=engine)

# Get the lowest id. 
min = s.query(func.min(Travel.id)).scalar()


In [13]:
# Print the budget for the lowest id.
print(min)

1


In [14]:
# Set the budget for the lowest id to 2500.
s.query(Travel).filter(Travel.id == min).update({'budget': 2500.0})

1

In [15]:
# Commit the updates to the database 
s.commit()

In [16]:
# Check that the budget has been updated.
# Collect all of the destinations and print their information
sq = s.query(Travel).all()
for _ in sq:
    print(_.id, _.city, _.country, _.distance, _.budget, _.visited, sep=', ')

1, Ubud, Indonesia, 10200.0, 2500.0, True
2, Swakopmund, Namibia, 7200.0, 5000.0, True
3, Ollantaytambo, Peru, 3600.0, 3000.0, True
4, Maastricht, Netherlands, 3900.0, 4000.0, True
5, Burlington, USA, 600.0, 200.0, True


In [17]:
s.close()